In [ ]:
from rdkit import Chem
from rdkit.Chem import rdShapeAlign
import numpy as np

def _random_rotation_matrix(rng: np.random.Generator) -> np.ndarray:
    # random unit quaternion -> rotation matrix
    u1, u2, u3 = rng.random(3)
    q1 = np.sqrt(1 - u1) * np.sin(2*np.pi*u2)
    q2 = np.sqrt(1 - u1) * np.cos(2*np.pi*u2)
    q3 = np.sqrt(u1)     * np.sin(2*np.pi*u3)
    q4 = np.sqrt(u1)     * np.cos(2*np.pi*u3)
    R = np.array([
        [1 - 2*(q3*q3 + q4*q4),     2*(q2*q3 - q1*q4),     2*(q2*q4 + q1*q3)],
        [    2*(q2*q3 + q1*q4), 1 - 2*(q2*q2 + q4*q4),     2*(q3*q4 - q1*q2)],
        [    2*(q2*q4 - q1*q3),     2*(q3*q4 + q1*q2), 1 - 2*(q2*q2 + q3*q3)]
    ], dtype=float)
    return R

def _apply_rigid_transform(mol: Chem.Mol, conf_id: int, R: np.ndarray, t: np.ndarray) -> None:
    conf = mol.GetConformer(conf_id)
    for i in range(mol.GetNumAtoms()):
        p = np.array(conf.GetAtomPosition(i), dtype=float)
        p2 = R @ p + t
        conf.SetAtomPosition(i, p2)

def align_molblocks_shape(
    ref_molblock: str,
    probe_molblock: str,
    ref_conf_id: int = -1,
    probe_conf_id: int = -1,
    n_starts: int = 25,
    use_colors: bool = True,
    seed: int = 0,
):
    ref = Chem.MolFromMolBlock(ref_molblock, removeHs=False)
    probe0 = Chem.MolFromMolBlock(probe_molblock, removeHs=False)
    if ref is None or probe0 is None:
        raise ValueError("Failed to parse one of the MolBlocks.")
    if ref.GetNumConformers() == 0 or probe0.GetNumConformers() == 0:
        raise ValueError("Both molecules must have 3D coordinates (at least one conformer).")

    # Speedup when aligning many probes: precompute the reference shape once
    ref_shape = rdShapeAlign.PrepareConformer(ref, confId=ref_conf_id)  # :contentReference[oaicite:2]{index=2}

    rng = np.random.default_rng(seed)
    best = (-1e9, None, None)  # (score, (shapeT, colorT), aligned_probe)

    # pick scoring objective
    def score(shapeT, colorT):
        return colorT if use_colors else shapeT

    for k in range(n_starts):
        probe = Chem.Mol(probe0)  # copy
        # random rigid-body pose as a different starting point
        R = _random_rotation_matrix(rng)
        t = rng.normal(scale=5.0, size=3)  # random translation (angstrom-ish scale)
        _apply_rigid_transform(probe, conf_id=probe_conf_id, R=R, t=t)

        shapeT, colorT = rdShapeAlign.AlignMol(
            ref_shape, probe,
            probeConfId=probe_conf_id,
            useColors=use_colors
        )

        sc = score(shapeT, colorT)
        if sc > best[0]:
            best = (sc, (shapeT, colorT), probe)

    _, (shapeT, colorT), aligned_probe = best
    return aligned_probe, shapeT, colorT

# ---- usage ----
ref_block = '\n     RDKit          3D\n\n 30 31  0  0  0  0  0  0  0  0999 V2000\n    3.7402    0.0397    0.8490 C   0  0  0  0  0  0  0  0  0  0  0  0\n    2.7987   -0.4620   -0.1298 N   0  0  0  0  0  0  0  0  0  0  0  0\n    3.3956   -0.4910   -1.4488 C   0  0  0  0  0  0  0  0  0  0  0  0\n    1.5521    0.2848   -0.1319 C   0  0  0  0  0  0  0  0  0  0  0  0\n    0.7236    0.0347    1.1357 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -0.6801    0.5037    0.9471 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -1.2422    1.6509    1.4228 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -2.5464    1.7483    1.0112 N   0  0  0  0  0  0  0  0  0  0  0  0\n   -2.8527    0.6566    0.2526 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -4.0346    0.3002   -0.3843 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -4.0470   -0.8838   -1.0893 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -2.9138   -1.6993   -1.1600 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -1.7396   -1.3516   -0.5314 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -1.6953   -0.1565    0.1865 C   0  0  0  0  0  0  0  0  0  0  0  0\n    4.0177    1.0917    0.6697 H   0  0  0  0  0  0  0  0  0  0  0  0\n    4.6440   -0.5674    0.8147 H   0  0  0  0  0  0  0  0  0  0  0  0\n    3.3112   -0.0435    1.8448 H   0  0  0  0  0  0  0  0  0  0  0  0\n    3.6664    0.5117   -1.8181 H   0  0  0  0  0  0  0  0  0  0  0  0\n    4.2969   -1.1023   -1.4179 H   0  0  0  0  0  0  0  0  0  0  0  0\n    2.6956   -0.9416   -2.1513 H   0  0  0  0  0  0  0  0  0  0  0  0\n    0.9681   -0.0544   -0.9912 H   0  0  0  0  0  0  0  0  0  0  0  0\n    1.7239    1.3707   -0.2507 H   0  0  0  0  0  0  0  0  0  0  0  0\n    1.1640    0.5551    1.9872 H   0  0  0  0  0  0  0  0  0  0  0  0\n    0.7336   -1.0378    1.3390 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -0.8067    2.4149    2.0388 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -3.1746    2.4994    1.2391 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -4.9111    0.9295   -0.3269 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -4.9493   -1.1916   -1.5973 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -2.9688   -2.6209   -1.7212 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -0.8693   -1.9882   -0.5881 H   0  0  0  0  0  0  0  0  0  0  0  0\n  1  2  1  0\n  2  3  1  0\n  2  4  1  0\n  4  5  1  0\n  5  6  1  0\n  6  7  2  0\n  7  8  1  0\n  8  9  1  0\n  9 10  2  0\n 10 11  1  0\n 11 12  2  0\n 12 13  1  0\n 13 14  2  0\n 14  6  1  0\n 14  9  1  0\n  1 15  1  0\n  1 16  1  0\n  1 17  1  0\n  3 18  1  0\n  3 19  1  0\n  3 20  1  0\n  4 21  1  0\n  4 22  1  0\n  5 23  1  0\n  5 24  1  0\n  7 25  1  0\n  8 26  1  0\n 10 27  1  0\n 11 28  1  0\n 12 29  1  0\n 13 30  1  0\nM  END\n'
probe_block = '\n     RDKit          3D\n\n 30 31  0  0  0  0  0  0  0  0999 V2000\n   -3.7192   -0.9253   -0.4447 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -2.8962    0.1138    0.1349 N   0  0  0  0  0  0  0  0  0  0  0  0\n   -3.5068    1.4144   -0.0406 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -1.5347    0.0895   -0.3691 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -0.7256   -1.0729    0.2227 C   0  0  0  0  0  0  0  0  0  0  0  0\n    0.7076   -0.9811   -0.1790 C   0  0  0  0  0  0  0  0  0  0  0  0\n    1.3332   -1.6916   -1.1609 C   0  0  0  0  0  0  0  0  0  0  0  0\n    2.6483   -1.3193   -1.2583 N   0  0  0  0  0  0  0  0  0  0  0  0\n    2.9059   -0.3517   -0.3352 C   0  0  0  0  0  0  0  0  0  0  0  0\n    4.0850    0.3270   -0.0478 C   0  0  0  0  0  0  0  0  0  0  0  0\n    4.0566    1.2662    0.9587 C   0  0  0  0  0  0  0  0  0  0  0  0\n    2.8842    1.5336    1.6698 C   0  0  0  0  0  0  0  0  0  0  0  0\n    1.7251    0.8559    1.3755 C   0  0  0  0  0  0  0  0  0  0  0  0\n    0.6042    1.1346    2.0784 F   0  0  0  0  0  0  0  0  0  0  0  0\n    1.6982   -0.1063    0.3661 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -4.7162   -0.8725   -0.0095 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -3.2983   -1.9007   -0.2113 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -3.8100   -0.8334   -1.5403 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -3.6016    1.7046   -1.1000 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -4.5014    1.4021    0.4037 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -2.9080    2.1638    0.4745 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -1.0537    1.0222   -0.0675 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -1.5055    0.0344   -1.4737 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -0.8175   -1.0250    1.3083 H   0  0  0  0  0  0  0  0  0  0  0  0\n   -1.1270   -2.0269   -0.1209 H   0  0  0  0  0  0  0  0  0  0  0  0\n    0.9317   -2.4522   -1.8027 H   0  0  0  0  0  0  0  0  0  0  0  0\n    3.3181   -1.7067   -1.9002 H   0  0  0  0  0  0  0  0  0  0  0  0\n    4.9913    0.1206   -0.5965 H   0  0  0  0  0  0  0  0  0  0  0  0\n    4.9537    1.8110    1.2097 H   0  0  0  0  0  0  0  0  0  0  0  0\n    2.8785    2.2720    2.4558 H   0  0  0  0  0  0  0  0  0  0  0  0\n  1  2  1  0\n  2  3  1  0\n  2  4  1  0\n  4  5  1  0\n  5  6  1  0\n  6  7  2  0\n  7  8  1  0\n  8  9  1  0\n  9 10  2  0\n 10 11  1  0\n 11 12  2  0\n 12 13  1  0\n 13 14  1  0\n 13 15  2  0\n 15  6  1  0\n 15  9  1  0\n  1 16  1  0\n  1 17  1  0\n  1 18  1  0\n  3 19  1  0\n  3 20  1  0\n  3 21  1  0\n  4 22  1  0\n  4 23  1  0\n  5 24  1  0\n  5 25  1  0\n  7 26  1  0\n  8 27  1  0\n 10 28  1  0\n 11 29  1  0\n 12 30  1  0\nM  END\n'

aligned_probe, shapeT, colorT = align_molblocks_shape(
    ref_block, probe_block, n_starts=30, use_colors=True, seed=123
)
print("shapeTanimoto:", shapeT, "colorTanimoto:", colorT)

In [ ]:
import py3Dmol
from rdkit import Chem

def overlay_molblocks_3d_display(ref_molblock: str, probe_molblock: str,
                                ref_color="cyan", probe_color="magenta"):
    view = py3Dmol.view(width=600, height=400)

    # Add reference as model 0
    view.addModel(ref_molblock, "sdf")
    view.setStyle({"model": 0}, {"stick": {"color": ref_color}})

    # Add aligned probe as model 1
    view.addModel(probe_molblock, "sdf")
    view.setStyle({"model": 1}, {"stick": {"color": probe_color}})

    view.zoomTo()
    view.show()

ref_m = Chem.MolFromMolBlock(ref_block, removeHs=False)
probe_block_aligned = Chem.MolToMolBlock(aligned_probe)

overlay_molblocks_3d_display(Chem.MolToMolBlock(ref_m), probe_block_aligned)